# Feature Engineering
This notebook will create a small set of robust, cross-dataset features derived from the canonical features.

## Goals
1. Normalize raw flow counts by time to capture traffic intensity (e.g., packets per second, bytes per second)
2. Capture flow density characteristics (e.g., average bytes per packet)
3. Encode directional asymmetry in traffic behavior (e.g., packet-level direction ratio, byte-level direction ratio)

In [7]:
!pip -q install "PyAthena[SQLAlchemy]" sqlalchemy s3fs

In [8]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# Display settings
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

## Connect to Athena

In [9]:
sess = sagemaker.Session()
region = boto3.Session().region_name

results_bucket = sess.default_bucket()
athena_results_path = f"s3://{results_bucket}/athena/staging/"

database_name = "aai540_eda"

engine = create_engine(
    f"awsathena+rest://@athena.{region}.amazonaws.com:443/{database_name}",
    connect_args={"s3_staging_dir": athena_results_path, "region_name": region},
)
print("Region:", region)
print("Athena results:", athena_results_path)

Region: us-east-1
Athena results: s3://sagemaker-us-east-1-128131109986/athena/staging/


In [10]:
# Helper functions for queries
def exec_ddl(sql: str):
    with engine.begin() as conn:
        conn.execute(text(sql))

def read_sql(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, engine)

## Verify merged dataset

In [15]:
merged_dataset = read_sql(f"""
SELECT *
FROM {database_name}.merged_canonical_normalized
Limit 5
""")
merged_dataset

,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,label,original_attack_type,attack_category,source_dataset
0,0.000127,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT
1,0.000001,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT
2,0.000010,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT
3,0.000066,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT
4,0.000020,2,0,1,1,0,0,1,dos,DoS/DDoS,TON_IoT


## Engineered Features
1. **`pkt_rate`** (*`pkt_rate = pkt_total / duration`*): Number of packets transmitted per second during the flow.
2. **`byte_rate`** (*`byte_rate = bytes_total / duration`*): Total bytes transferred per second during the flow.
3. **`bytes_per_pkt`** (*`bytes_per_pkt = bytes_total / (pkt_total + 1)`*): Average number of bytes carried per packet in the flow.
4. **`pkt_ratio`** (*`pkt_ratio = pkt_fwd / (pkt_bwd + 1)`*): Ratio of forward packets to backward packets in the flow.
5. **`byte_ratio`** (*`byte_ratio = bytes_fwd / (bytes_bwd + 1)`*): Ratio of forward bytes to backward bytes in the flow.

## **`pkt_rate`** (*`pkt_rate = pkt_total / duration`*)

This creates a new table version: `merged_canonical_normalized_v1`

In [21]:
# drop table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v1")

# create table pkt_rate
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v1
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v1/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN duration IS NULL OR duration <= 0 THEN NULL
    ELSE CAST(pkt_total AS DOUBLE) / CAST(duration AS DOUBLE)
  END AS pkt_rate
FROM {database_name}.merged_canonical_normalized
""")

In [34]:
read_sql(f"""
SELECT duration, pkt_total, pkt_rate
FROM {database_name}.merged_canonical_normalized_v1
pkt_total
Limit 25
""")

,duration,pkt_total,pkt_rate
0,0.316483,13,41.076456
1,0.228041,8,35.081411
2,1.113813,17,15.262885
3,0.403528,11,27.259571
4,0.400382,11,27.473763
5,0.445936,10,22.424743
6,0.290839,9,30.944956
7,0.382943,11,28.724902
8,0.276969,10,36.105124
9,0.268739,9,33.489743


### Sanity check

In [23]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN duration IS NULL OR duration <= 0 THEN 1 ELSE 0 END) AS bad_duration_rows,
  SUM(CASE WHEN (duration IS NULL OR duration <= 0) AND pkt_rate IS NOT NULL THEN 1 ELSE 0 END) AS pkt_rate_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v1
""")


,rows_total,bad_duration_rows,pkt_rate_should_be_null_but_isnt
0,26708942,5325854,0


## **`byte_rate`** (*`byte_rate = bytes_total / duration`*)

This creates a new table version: `merged_canonical_normalized_v2`

In [28]:
# drop v2 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v2")

# Create v2 with byte_rate (built on v1 so pkt_rate is retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v2
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v2/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN duration IS NULL OR duration <= 0 THEN NULL
    ELSE CAST(bytes_total AS DOUBLE) / CAST(duration AS DOUBLE)
  END AS byte_rate
FROM {database_name}.merged_canonical_normalized_v1
""")

In [31]:
read_sql(f"""
SELECT duration, bytes_total, byte_rate, pkt_total, pkt_rate
FROM {database_name}.merged_canonical_normalized_v2
WHERE duration IS NOT NULL
LIMIT 25
""")

,duration,bytes_total,byte_rate,pkt_total,pkt_rate
0,0.000201,324,1.611940e+06,4,19900.497512
1,0.023868,0,0.000000e+00,2,83.794201
2,0.063017,248,3.935446e+03,4,63.474935
3,0.023855,196,8.216307e+03,4,167.679732
4,26.168369,0,0.000000e+00,2,0.076428
5,0.095071,0,0.000000e+00,2,21.036909
6,26.152456,0,0.000000e+00,2,0.076475
7,0.000060,12,2.000000e+05,2,33333.333333
8,26.148512,0,0.000000e+00,2,0.076486
9,0.057114,0,0.000000e+00,2,35.017684


### Sanity check

In [32]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN duration IS NULL OR duration <= 0 THEN 1 ELSE 0 END) AS bad_duration_rows,
  SUM(CASE WHEN (duration IS NULL OR duration <= 0) AND byte_rate IS NOT NULL THEN 1 ELSE 0 END) AS byte_rate_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v2
""")


,rows_total,bad_duration_rows,byte_rate_should_be_null_but_isnt
0,26708942,5325854,0


## **`bytes_per_pkt`** (*`bytes_per_pkt = bytes_total / (pkt_total + 1)`*)

This creates a new table version: `merged_canonical_normalized_v3`

In [35]:
# drop v3 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v3")

# create v3 with bytes_per_pkt (built on v2 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v3
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v3/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN pkt_total IS NULL OR pkt_total <= 0 THEN NULL
    ELSE CAST(bytes_total AS DOUBLE) / CAST(pkt_total AS DOUBLE)
  END AS bytes_per_pkt
FROM {database_name}.merged_canonical_normalized_v2
""")

In [48]:
read_sql(f"""
SELECT duration, pkt_total, bytes_total, bytes_per_pkt
FROM {database_name}.merged_canonical_normalized_v3
WHERE bytes_total > 0
LIMIT 25
""")

,duration,pkt_total,bytes_total,bytes_per_pkt
0,0.003623,6,386,64.333333
1,4.154148,16,5116,319.750000
2,3.264887,20,2468,123.400000
3,0.286771,2,853,426.500000
4,0.289019,4,473,118.250000
5,0.002456,11,3324,302.181818
6,0.011957,10,1182,118.200000
7,0.000523,2,573,286.500000
8,0.739264,22,4265,193.863636
9,0.001237,2,573,286.500000


### Sanity check

In [38]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN pkt_total IS NULL OR pkt_total <= 0 THEN 1 ELSE 0 END) AS bad_pkt_total_rows,
  SUM(CASE WHEN (pkt_total IS NULL OR pkt_total <= 0) AND bytes_per_pkt IS NOT NULL THEN 1 ELSE 0 END) AS bytes_per_pkt_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v3
""")

,rows_total,bad_pkt_total_rows,bytes_per_pkt_should_be_null_but_isnt
0,26708942,115260,0


## **`pkt_ratio`** (*`pkt_ratio = pkt_fwd / (pkt_bwd + 1)`*)

This creates a new table version: `merged_canonical_normalized_v4`

In [39]:
# drop v4 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v4")

# create v4 with pkt_ratio (built on v3 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v4
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v4/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN pkt_fwd IS NULL OR pkt_fwd < 0 THEN NULL
    WHEN pkt_bwd IS NULL OR pkt_bwd < 0 THEN NULL
    ELSE CAST(pkt_fwd AS DOUBLE) / (CAST(pkt_bwd AS DOUBLE) + 1.0)
  END AS pkt_ratio
FROM {database_name}.merged_canonical_normalized_v3
""")

In [47]:
read_sql(f"""
SELECT duration, pkt_total, pkt_fwd, pkt_bwd, pkt_ratio
FROM {database_name}.merged_canonical_normalized_v4
LIMIT 25
""")

,duration,pkt_total,pkt_fwd,pkt_bwd,pkt_ratio
0,60.709581,5,3,2,1.0
1,0.000001,2,1,1,0.5
2,0.000220,2,1,1,0.5
3,0.000059,2,1,1,0.5
4,60.724468,5,3,2,1.0
5,0.000006,2,1,1,0.5
6,60.710659,5,3,2,1.0
7,0.007426,2,1,1,0.5
8,60.724567,5,3,2,1.0
9,60.730921,5,3,2,1.0


## **`byte_ratio`** (*`byte_ratio = bytes_fwd / (bytes_bwd + 1)`*)

This creates a new table version: `merged_canonical_normalized_v5`

In [49]:
# drop v5 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v5")

# Create v5 with byte_ratio (built on v4 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v5
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v5/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN bytes_fwd IS NULL OR bytes_fwd < 0 THEN NULL
    WHEN bytes_bwd IS NULL OR bytes_bwd < 0 THEN NULL
    ELSE CAST(bytes_fwd AS DOUBLE) / (CAST(bytes_bwd AS DOUBLE) + 1.0)
  END AS byte_ratio
FROM {database_name}.merged_canonical_normalized_v4
""")

In [51]:
read_sql(f"""
SELECT
  bytes_fwd,
  bytes_bwd,
  byte_ratio
FROM {database_name}.merged_canonical_normalized_v5
LIMIT 25
""")

,bytes_fwd,bytes_bwd,byte_ratio
0,1684,10168,0.165601
1,37202,3380,11.003253
2,2062,2308,0.893027
3,37202,3380,11.003253
4,34916,1546546,0.022577
5,1580,10168,0.155374
6,146,178,0.815642
7,2230,15258,0.146143
8,544,304,1.783607
9,2854,29272,0.097496


In [ ]:
## Finalize Feature Engineering Table

In [52]:
# drop final table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.feature_engineered")

# create final feature_engineered table from v5
exec_ddl(f"""
CREATE TABLE {database_name}.feature_engineered
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/feature_engineered/',
  parquet_compression = 'SNAPPY'
) AS
SELECT *
FROM {database_name}.merged_canonical_normalized_v5
""")


In [53]:
read_sql(f"""
SELECT
  pkt_rate,
  byte_rate,
  bytes_per_pkt,
  pkt_ratio,
  byte_ratio
FROM {database_name}.feature_engineered
LIMIT 10
""")

,pkt_rate,byte_rate,bytes_per_pkt,pkt_ratio,byte_ratio
0,4077.393545,1.649267e+06,404.490566,0.945455,0.080161
1,7349.762214,1.623649e+06,220.911765,0.971429,0.174314
2,26.038514,1.632615e+03,62.700000,0.909091,0.939722
3,88.852558,4.108542e+04,462.400000,2.000000,27.813084
4,96.467612,4.072902e+04,422.204082,1.200000,10.872884
5,222222.222222,2.933333e+07,132.000000,2.000000,264.000000
6,6052.222030,1.307799e+06,216.085714,0.918919,0.172908
7,3686.635945,2.986175e+05,81.000000,0.666667,0.815642
8,4385.964912,6.074561e+05,138.500000,0.923077,0.470588
9,6040.673871,1.804148e+06,298.666667,0.936170,0.114700


## Clean Athena catalog

In [54]:
tables_to_drop = [
    "merged_canonical_normalized_v1",
    "merged_canonical_normalized_v2",
    "merged_canonical_normalized_v3",
    "merged_canonical_normalized_v4",
    "merged_canonical_normalized_v5",
]

for t in tables_to_drop:
    exec_ddl(f"DROP TABLE IF EXISTS {database_name}.{t}")


In [55]:
invalid_rows = read_sql("""
SELECT *
FROM aai540_eda.feature_engineered
WHERE
    duration IS NULL
 OR pkt_total IS NULL
 OR bytes_total IS NULL
 OR pkt_fwd IS NULL
 OR pkt_bwd IS NULL
 OR bytes_fwd IS NULL
 OR bytes_bwd IS NULL

 OR duration < 0
 OR pkt_total < 0
 OR bytes_total < 0
 OR pkt_fwd < 0
 OR pkt_bwd < 0
 OR bytes_fwd < 0
 OR bytes_bwd < 0
LIMIT 100
""")

invalid_rows


,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,label,original_attack_type,attack_category,source_dataset,pkt_rate,byte_rate,bytes_per_pkt,pkt_ratio,byte_ratio
0,-0.000001,2,12,1,1,6,6,0,BENIGN,Normal,CIC-IDS2017,None,None,6.0,0.5,0.857143
1,-0.000001,2,6,1,1,6,0,0,BENIGN,Normal,CIC-IDS2017,None,None,3.0,0.5,6.000000
2,-0.000001,2,6,1,1,6,0,0,BENIGN,Normal,CIC-IDS2017,None,None,3.0,0.5,6.000000
3,-0.000001,2,6,1,1,6,0,0,BENIGN,Normal,CIC-IDS2017,None,None,3.0,0.5,6.000000
4,-0.000001,2,12,1,1,6,6,0,BENIGN,Normal,CIC-IDS2017,None,None,6.0,0.5,0.857143
5,-0.000001,2,12,1,1,6,6,0,BENIGN,Normal,CIC-IDS2017,None,None,6.0,0.5,0.857143
6,-0.000001,2,0,1,1,0,0,0,BENIGN,Normal,CIC-IDS2017,None,None,0.0,0.5,0.000000
7,-0.000001,2,0,1,1,0,0,0,BENIGN,Normal,CIC-IDS2017,None,None,0.0,0.5,0.000000
8,-0.000001,2,6,1,1,6,0,0,BENIGN,Normal,CIC-IDS2017,None,None,3.0,0.5,6.000000
9,-0.000001,2,12,1,1,6,6,0,BENIGN,Normal,CIC-IDS2017,None,None,6.0,0.5,0.857143
